In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier 
import shap
import matplotlib.pyplot as plt

In [2]:
df= pd.read_csv('speed_dating_data.csv')
df

,gender,age,income,goal,career,dec,attr,sinc,intel,fun,amb,shar,like,prob,met
0,0,21.0,69487.0,2.0,lawyer,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0
1,0,21.0,69487.0,2.0,lawyer,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0
2,0,21.0,69487.0,2.0,lawyer,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0
3,0,21.0,69487.0,2.0,lawyer,1,7.0,6.0,8.0,7.0,6.0,8.0,7.0,6.0,2.0
4,0,21.0,69487.0,2.0,lawyer,1,5.0,6.0,7.0,7.0,6.0,6.0,6.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8373,1,25.0,NaN,1.0,assistant master of the universe (otherwise it...,0,3.0,5.0,5.0,5.0,NaN,NaN,2.0,5.0,0.0
8374,1,25.0,NaN,1.0,assistant master of the universe (otherwise it...,0,4.0,6.0,8.0,4.0,4.0,NaN,4.0,4.0,0.0
8375,1,25.0,NaN,1.0,assistant master of the universe (otherwise it...,0,4.0,7.0,8.0,8.0,8.0,NaN,6.0,5.0,0.0
8376,1,25.0,NaN,1.0,assistant master of the universe (otherwise it...,0,4.0,6.0,5.0,4.0,NaN,5.0,5.0,5.0,0.0


In [3]:
df.columns #target: dec, features: income, attr, sinc, intel, fun, amb, 

Index(['gender', 'age', 'income', 'goal', 'career', 'dec', 'attr', 'sinc',
       'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met'],
      dtype='object')

In [7]:
df['career'].unique()

array(['lawyer', 'law', 'Economist', 'lawyer/policy work', 'Law',
       'Journalist', 'Congresswoman, and comedian',
       'To create early childhood intervention programs',
       'Academia, Research, Banking, Life', 'Corporate Lawyer', 'Lawyer',
       'Corporate attorney', 'research/financial industry',
       'academics or journalism', 'Financial Services', 'ceo', 'CEO',
       'Undecided', 'Informatics', 'psychologist',
       'health/nutrition oriented social worker', 'Social Worker',
       'Social work with children', 'Speech Language Pathologist', nan,
       'Social Work Administration', 'Professor', "Clidren's TV",
       'Banking', 'Capital Markets', 'Biostatistics',
       'Organizational Change Consultant', 'tech professional',
       'Engineer', 'Academic', 'academia', 'banker / academia', 'banker',
       'Music production', 'Entrepreneur',
       'Intellectual Property Attorney', 'Medicine', 'consulting',
       'LAWYER', 'social worker', 'comedienne', 'attorney',
  

In [9]:
len(df['career'].unique())

368

In [11]:
df.shape

(8378, 15)

In [13]:
df.dtypes

gender      int64
age       float64
income    float64
goal      float64
career     object
dec         int64
attr      float64
sinc      float64
intel     float64
fun       float64
amb       float64
shar      float64
like      float64
prob      float64
met       float64
dtype: object

In [15]:
df['id'] = df.set_index(['gender', 'age', 'income', 'career']).index.factorize()[0]+1

In [17]:
# dropping cols that are partner-dependent or have too many missings
df = df.drop(columns=['income', 'prob', 'shar', 'met', 'career'])

In [19]:
average_scores = df.groupby('id')[['attr', 'sinc', 'intel', 'fun', 'amb', 'like']].mean().reset_index()
df = df.groupby('id').first()
df = df.drop(columns = ['attr', 'sinc', 'intel', 'fun', 'amb', 'like']).join(average_scores.set_index('id'), on = 'id')
df

,gender,age,goal,dec,attr,sinc,intel,fun,amb,like
id,,,,,,,,,,
1,0,21.0,2.0,1,5.700000,7.300000,7.300000,6.800000,6.300000,6.500000
2,0,24.0,1.0,0,6.400000,7.000000,7.700000,6.100000,6.500000,6.600000
3,0,25.0,6.0,0,8.100000,8.600000,9.400000,7.700000,8.800000,8.200000
4,0,23.0,1.0,0,6.400000,8.900000,8.600000,7.800000,7.800000,6.600000
5,0,21.0,2.0,0,6.300000,6.000000,7.000000,6.000000,5.600000,7.200000
...,...,...,...,...,...,...,...,...,...,...
535,1,30.0,1.0,1,4.476190,6.714286,5.809524,4.476190,3.904762,5.047619
536,1,28.0,1.0,0,4.590909,7.045455,6.909091,6.727273,6.863636,5.363636
537,1,30.0,2.0,0,5.636364,5.954545,5.772727,5.500000,5.045455,5.500000


In [21]:
df.isna().sum()

gender    0
age       4
goal      3
dec       0
attr      0
sinc      1
intel     1
fun       1
amb       1
like      0
dtype: int64

In [23]:
df= df.dropna()

In [31]:
target = df['dec']
df= df.drop(columns= ['dec'])
features = df.select_dtypes(include=['number'])
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [33]:
df

,gender,age,goal,attr,sinc,intel,fun,amb,like
id,,,,,,,,,
1,0,21.0,2.0,5.700000,7.300000,7.300000,6.800000,6.300000,6.500000
2,0,24.0,1.0,6.400000,7.000000,7.700000,6.100000,6.500000,6.600000
3,0,25.0,6.0,8.100000,8.600000,9.400000,7.700000,8.800000,8.200000
4,0,23.0,1.0,6.400000,8.900000,8.600000,7.800000,7.800000,6.600000
5,0,21.0,2.0,6.300000,6.000000,7.000000,6.000000,5.600000,7.200000
...,...,...,...,...,...,...,...,...,...
535,1,30.0,1.0,4.476190,6.714286,5.809524,4.476190,3.904762,5.047619
536,1,28.0,1.0,4.590909,7.045455,6.909091,6.727273,6.863636,5.363636
537,1,30.0,2.0,5.636364,5.954545,5.772727,5.500000,5.045455,5.500000


In [35]:
normalizer = MinMaxScaler()

In [37]:
normalizer.fit(X_train)  

MinMaxScaler()

In [39]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [41]:
X_train_norm 

array([[1.        , 0.28571429, 0.2       , ..., 0.63561077, 0.68640351,
        0.69330855],
       [1.        , 0.47619048, 0.        , ..., 0.55279503, 0.54605263,
        0.45167286],
       [0.        , 0.28571429, 0.        , ..., 0.59006211, 0.52236842,
        0.65241636],
       ...,
       [1.        , 0.33333333, 0.4       , ..., 0.49245785, 0.49686717,
        0.38077536],
       [1.        , 0.52380952, 0.        , ..., 0.50807453, 0.39210526,
        0.52973978],
       [1.        , 0.19047619, 0.8       , ..., 0.52173913, 0.61687307,
        0.62515489]])

In [43]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_train_norm.head()

,gender,age,goal,attr,sinc,intel,fun,amb,like
0,1.0,0.285714,0.2,0.653704,0.607966,0.620690,0.635611,0.686404,0.693309
1,1.0,0.476190,0.0,0.338889,0.591195,0.531034,0.552795,0.546053,0.451673
2,0.0,0.285714,0.0,0.633333,0.640252,0.605517,0.590062,0.522368,0.652416
3,0.0,0.142857,0.2,0.592593,0.930818,0.856705,0.468599,0.921053,0.625155
4,1.0,0.190476,1.0,0.388889,0.631027,0.477395,0.407867,0.388889,0.338910


In [45]:
knn = KNeighborsClassifier(n_neighbors=5)

In [47]:
knn.fit(X_train_norm, y_train)

KNeighborsClassifier()

In [49]:
pred = knn.predict(X_test_norm)  
pred

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1])

In [51]:
y_test.values

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0])

In [53]:
knn.score(X_test_norm, y_test)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


0.5607476635514018

In [59]:
explainer = shap.KernelExplainer(knn.predict_proba, X_train)
shap_values = explainer.shap_values(X_test_norm[:9])
# Save the standard summary plot
plt.figure()
shap.summary_plot(shap_values, X_test_norm[:9], show=False)  # Set show=False to prevent displaying inline
plt.savefig("shap_summary_plot.png", dpi=300, bbox_inches='tight')
plt.close()  # Close the plot to reset the figure
# Save the bar summary plot
plt.figure()
shap.summary_plot(shap_values, X_test_norm[:9], plot_type="bar", show=False)  # Bar plot type
plt.savefig("shap_summary_plot_bar.png", dpi=300, bbox_inches='tight')
plt.close()
# Save the customized summary plot with larger plot size
plt.figure()
shap.summary_plot(shap_values, X_test_norm[:9], plot_size=(10, 6), show=False)  # Set custom plot size
plt.savefig("shap_summary_plot_custom_size.png", dpi=300, bbox_inches='tight')
plt.close()


Using 424 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/9 [00:00<?, ?it/s]

NameError: name 'X' is not defined

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [61]:
print(X_test.columns)

Index(['gender', 'age', 'goal', 'attr', 'sinc', 'intel', 'fun', 'amb', 'like'], dtype='object')


In [63]:
for i, feature_name in enumerate(X_test.columns):
    print(f"{feature_name}: {shap_values[i].mean()}")

gender: -1.5651060694368527e-16
age: -1.58823571578321e-16
goal: -1.553541246263674e-16
attr: -1.5728159515523052e-16
sinc: -1.5573961873214e-16
intel: -1.5728159515523052e-16
fun: -1.5573961873214e-16
amb: -1.5419764230904951e-16
like: -1.5496863052059476e-16
